# Build the Neural Network

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get device for training

In [2]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## Define a simple neural network class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

To use the `model`, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [5]:
X = torch.rand(1, 28, 28, device=device) # Data
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


## Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

### nn.Linear

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)

In [10]:
hidden1=layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

### nn.ReLU

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1678,  0.4237, -0.1678, -0.4029, -0.0675,  0.0747, -0.1694,  0.1210,
          0.0478,  0.3904,  0.0263,  0.2688,  0.1927, -0.2837, -0.2937,  0.2639,
         -0.2293, -0.0790, -0.2320, -0.3794],
        [ 0.4222,  0.4228,  0.0315, -0.3446, -0.2184,  0.0151, -0.1590,  0.1061,
          0.1150, -0.0406, -0.0021, -0.0126,  0.3751, -0.5129, -0.1584,  0.3447,
         -0.4979, -0.3919, -0.3094, -0.3482],
        [ 0.5440,  0.5292, -0.0225, -0.6428,  0.1135,  0.0344, -0.2962,  0.0739,
          0.0928,  0.1434,  0.1442, -0.0859,  0.1816, -0.5715, -0.5585,  0.3765,
         -0.3250, -0.0318,  0.3183, -0.4077]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1678, 0.4237, 0.0000, 0.0000, 0.0000, 0.0747, 0.0000, 0.1210, 0.0478,
         0.3904, 0.0263, 0.2688, 0.1927, 0.0000, 0.0000, 0.2639, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.4222, 0.4228, 0.0315, 0.0000, 0.0000, 0.0151, 0.0000, 0.1061, 0.1150,
         0.0000, 0.0000, 0.0000, 0.3751, 0.0000, 0.00

### nn.Sequential

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model parameters

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0148,  0.0309, -0.0091,  ...,  0.0222, -0.0152, -0.0270],
        [ 0.0174, -0.0188, -0.0295,  ...,  0.0191, -0.0299, -0.0063]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0138, 0.0012], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0140, -0.0034,  0.0149,  ...,  0.0351,  0.0293,  0.0056],
        [ 0.0076,  0.0375, -0.0140,  ..., -0.0029,  0.0255,  0.0091]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si

## Further reading
[torch.nn API](https://pytorch.org/docs/stable/nn.html)